# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [21]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
import gradio as gr
from typing import List, Tuple

In [22]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# Gemini API setup
google_api_key = os.getenv('GOOGLE_API_KEY')
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")


# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyCQ


In [23]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [24]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

def transform_message_for_gemini(message):
    return {
        "role": "user" if message["role"] == "user" else "model", # Map your "assistant" to "model"
        "parts": [
            {
                "text": message["content"]
            }
        ]
    }

def append_history_to_chat_data(existing_chat_history, history_response):
    for message in history_response:
        transformed_message = transform_message_for_gemini(message)
        existing_chat_history.append(transformed_message)
    return existing_chat_history

def chat_Gemini(message, history):
   
   # 1. Initialize your chat_history in the Gemini API format (empty for a new chat)
    chat_history = []
    updated_chat_history = append_history_to_chat_data(chat_history, history)

    # 2. Append the new user message to the chat history
    updated_chat_history.append({
        "role": "user",
        "parts": [
            {
                "text": message
            }
        ]
    })
    #1 Create an instance of the GenerativeModel class
    model_instance = genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        system_instruction=system_message
    )
    stream = model_instance.generate_content(contents=updated_chat_history, stream=True)
    #3 Iterate over the stream to get the response
    full_response_text = ""

    for chunk in stream:
        if chunk.text:
            print(chunk.text, end='') 
            full_response_text += chunk.text
            yield full_response_text

#gr.ChatInterface(fn=chat, type="messages").launch()
gr.ChatInterface(fn=chat_Gemini, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [25]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [26]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [35]:
# There's a particular dictionary structure that's required to describe our function:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# This is the function the Gemini model will be able to call.
def price_function_2(item: str) -> str:
    """
    Get the price of a given item.
    Args:
        item: The name of the item for which to find the price.
    """
    print(f"Tool 'price_function' called with item: {item}")
    if "shirt" in item.lower():
        return "The price of a shirt is $25."
    elif "jeans" in item.lower():
        return "The price of jeans is $50."
    else:
        return f"Sorry, I don't have a price for {item}."

In [36]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

# Tools for Gemini
tools_For_Gemini = [price_function_2]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

# --- Helper Function for History ---
def convert_history_to_gemini_format(history: List[Tuple[str, str]]) -> List[dict]:
    """
    Converts Gradio's chat history format to Gemini's format.
    Gradio: [("user message", "model message"), ...]
    Gemini: [{"role": "user", ...}, {"role": "model", ...}, ...]
    """
    gemini_history = []
    for user_msg, model_msg in history:
        gemini_history.append({"role": "user", "parts": [{"text": user_msg}]})
        gemini_history.append({"role": "model", "parts": [{"text": model_msg}]})
    return gemini_history

def chat_Gemini_with_Tool(message: str, history: List[Tuple[str, str]]):
    """
    Handles the chat interaction with the Gemini model, including tool calls.
    """
    # 1. Initialize the Gemini model
    # We pass the tools directly to the model during initialization.
    model_instance = genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        system_instruction=system_message,
        tools=tools_For_Gemini
    )

    # 2. Convert and build the chat history for the API call
    gemini_history = convert_history_to_gemini_format(history)
    gemini_history.append({"role": "user", "parts": [{"text": message}]})

    # 3. Call the model
    # We set stream=False because we need the full response to check for tool calls.
    # If a tool is called, we must execute it and send the result back before
    # we can get the final text response to show the user.
    print("Sending request to Gemini...")
    response = model_instance.generate_content(
        gemini_history,
        tool_config={"function_calling_config": "any"}
    )

    # 4. Check if the model's response includes a tool call
    candidate = response.candidates[0]
    if candidate.finish_reason == 'TOOL_CALLS':
        print("Gemini responded with a tool call.")
        # The model wants to call a function.
        # For this example, we'll process the first tool call.
        tool_call = candidate.content.parts[0].function_call
        tool_name = tool_call.name

        if tool_name == 'price_function_2':
            # A. Extract arguments and call the actual Python function
            args = {key: value for key, value in tool_call.args.items()}
            tool_result = price_function_2(**args)

            # B. Append the model's tool call and our tool's response to the history
            gemini_history.append(candidate.content)  # Add model's request to history
            gemini_history.append({
                "role": "tool",
                "parts": [{
                    "function_response": {
                        "name": "price_function_2",
                        "response": {"result": tool_result}
                    }
                }]
            })

            # C. Send the tool response back to the model to get a final answer
            print("Sending tool result back to Gemini...")
            final_response = model_instance.generate_content(gemini_history)
            return final_response.text
        else:
            # Handle cases where the model calls a function you haven't defined
            return f"Error: Model tried to call an unknown function: {tool_name}"
    else:
        # 5. If no tool call, just return the model's text response
        print("Gemini responded with text.")
        return response.text

In [ ]:
#gr.ChatInterface(fn=chat, type="messages").launch()
gr.ChatInterface(fn=chat_Gemini_with_Tool, type="messages").launch()

In [ ]:
import os
import gradio as gr
import google.generativeai as genai
from typing import List, Tuple

# --- Configuration ---
# It's best practice to use environment variables for API keys.
# Make sure to set your GOOGLE_API_KEY in your environment.
try:
    genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
except KeyError:
    print("ERROR: Please set the GOOGLE_API_KEY environment variable.")
    exit()

# --- Tool Definition ---
# This is the function the Gemini model will be able to call.
def price_function(item: str) -> str:
    """
    Get the price of a given item.
    Args:
        item: The name of the item for which to find the price.
    """
    print(f"Tool 'price_function' called with item: {item}")
    if "shirt" in item.lower():
        return "The price of a shirt is $25."
    elif "jeans" in item.lower():
        return "The price of jeans is $50."
    else:
        return f"Sorry, I don't have a price for {item}."

# --- Model and Chat Configuration ---
system_message = (
    "You are a helpful shopping assistant. "
    "You can use tools to find the price of items. "
    "When a user asks for a price, use the provided tool."
)

# This is the list of tools you provide to the model.
tools_For_Gemini = [price_function]

# --- Helper Function for History ---
def convert_history_to_gemini_format(history: List[Tuple[str, str]]) -> List[dict]:
    """
    Converts Gradio's chat history format to Gemini's format.
    Gradio: [("user message", "model message"), ...]
    Gemini: [{"role": "user", ...}, {"role": "model", ...}, ...]
    """
    gemini_history = []
    for user_msg, model_msg in history:
        gemini_history.append({"role": "user", "parts": [{"text": user_msg}]})
        gemini_history.append({"role": "model", "parts": [{"text": model_msg}]})
    return gemini_history

# --- Core Chat Logic ---
def chat_Gemini_with_Tool(message: str, history: List[Tuple[str, str]]):
    """
    Handles the chat interaction with the Gemini model, including tool calls.
    """
    # 1. Initialize the Gemini model
    # We pass the tools directly to the model during initialization.
    model_instance = genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        system_instruction=system_message,
        tools=tools_For_Gemini
    )

    # 2. Convert and build the chat history for the API call
    gemini_history = convert_history_to_gemini_format(history)
    gemini_history.append({"role": "user", "parts": [{"text": message}]})

    # 3. Call the model (without streaming) to check for tool calls
    print("Sending request to Gemini...")
    response = model_instance.generate_content(
        gemini_history,
        tool_config={"function_calling_config": "any"}
    )

    # 4. Check if the model's response includes a tool call
    candidate = response.candidates[0]
    if candidate.finish_reason == 'TOOL_CALLS':
        print("Gemini responded with a tool call.")
        # The model wants to call a function.
        tool_call = candidate.content.parts[0].function_call
        tool_name = tool_call.name

        if tool_name == 'price_function':
            # A. Extract arguments and call the actual Python function
            args = {key: value for key, value in tool_call.args.items()}
            tool_result = price_function(**args)

            # B. Append the model's tool call and our tool's response to the history
            gemini_history.append(candidate.content)  # Add model's request to history
            gemini_history.append({
                "role": "tool",
                "parts": [{
                    "function_response": {
                        "name": "price_function",
                        "response": {"result": tool_result}
                    }
                }]
            })

            # C. Send the tool response back to the model to get a final answer
            print("Sending tool result back to Gemini...")
            final_response = model_instance.generate_content(gemini_history)
            return final_response.text # This will now be text
        else:
            return f"Error: Model tried to call an unknown function: {tool_name}"
    else:
        # 5. If no tool call, just return the model's text response
        print("Gemini responded with text.")
        return response.text # This is safe because finish_reason was not TOOL_CALLS

# --- Gradio UI ---
if __name__ == "__main__":
    print("Starting Gradio interface...")
    iface = gr.ChatInterface(
        fn=chat_Gemini_with_Tool,
        title="Gemini Shopping Assistant",
        description="Ask me for the price of a shirt or jeans!",
        examples=[["How much is a shirt?"], ["What is the price of a pair of jeans?"]]
    )
    iface.launch()
    print("Gradio interface running.")



Starting Gradio interface...


c:\Users\hoang\.conda\envs\llms\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


Gradio interface running.


Sending request to Gemini...
Gemini finished with reason: 1. Attempting to return text if available.


Traceback (most recent call last):
  File "c:\Users\hoang\.conda\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hoang\.conda\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hoang\.conda\envs\llms\Lib\site-packages\gradio\blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hoang\.conda\envs\llms\Lib\site-packages\gradio\blocks.py", line 1729, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hoang\.conda\envs\llms\Lib\site-packages\gradio\utils.py", line 861, in async_wrapper
    response = await f(*args, **kwargs)
               ^^^^^^^^^^^^^^